In [ ]:
#Final Project Script  
#Agentic AI for news sentiment analysis and stock investment
#MODEL/CODE INSPO: https://python.langchain.com/docs/tutorials/agents/

In [ ]:
%pip install -U langgraph langchain-tavily langgraph-checkpoint-sqlite
pip install -U langgraph "langchain[openai]"
pip install -qU "langchain-xai"
pip install yfinance

In [38]:
#import packages
import pandas as pd
import numpy as np
from langchain_tavily import TavilySearch
from langchain.chat_models import init_chat_model
from langgraph.prebuilt import create_react_agent
import yfinance as yf
from langgraph.checkpoint.memory import MemorySaver
import getpass
import os
import json

In [28]:
#define finance function for agent
def yahoofinance(ticker: str) -> str:
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        return (
            f"Stock: {info.get('longName', 'N/A')} ({ticker})\n"
            f"Current Price: ${info.get('currentPrice', 'N/A')}\n"
            f"Market Cap: ${info.get('marketCap', 'N/A'):,}\n"
            f"52-Week High: ${info.get('fiftyTwoWeekHigh', 'N/A')}\n"
            f"52-Week Low: ${info.get('fiftyTwoWeekLow', 'N/A')}\n"
            f"P/E Ratio: {info.get('trailingPE', 'N/A')}"
        )
    except Exception as e:
        return f"Error fetching data for {ticker}: {str(e)}"

In [ ]:
# LANGSMITH API KEY: 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
# TAVILY API KEY: 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
# xAI API KEY: xai-yBadpNpX5cM417CcncvNiR2Kus6l499vHkqRxuWL013o4R3yey7NLclyFpDnXBbHg1ARv7dVWMCByuhG

In [30]:
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()
os.environ["TAVILY_API_KEY"] = getpass.getpass()
if not os.environ.get("XAI_API_KEY"):
  os.environ["XAI_API_KEY"] = getpass.getpass("Enter API key for xAI: ")

In [ ]:
#create Tavily search tool
search = TavilySearch(max_results=2)

In [ ]:
#define tools
tools = [search, yahoofinance]

In [49]:
#test tools
yahoo_finance_results = yahoofinance("AAPL")
print(yahoo_finance_results)

search_results = search.invoke("What is the weather in SF")
results = search_results['results']
info = results[1]
print(info.get('title'))
SF = info.get('content')
print(SF[0:149])




Stock: Apple Inc. (AAPL)
Current Price: $255.46
Market Cap: $3,791,126,003,712
52-Week High: $260.1
52-Week Low: $169.21
P/E Ratio: 38.764793
Weather in San Francisco in September 2025
* Weather * Weather in San Francisco # Weather in San Francisco in September 2025 * +70° +70° +63° +70° +63° +72° +63° +72° +64° +73° +63° +73° +63° 


In [ ]:
#integrate chatbot 
model = init_chat_model("grok-2", model_provider="xai")

In [17]:
#test chat bot
query = "Hi!"
response = model.invoke([{"role": "user", "content": query}])
response.text()

In [ ]:
#add tools
model_with_tools = model.bind_tools(tools)

In [ ]:
query = "Search for the weather in SF"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

In [ ]:
memory = MemorySaver()

In [20]:
agent_executor = create_react_agent(model, tools)
model_with_tools = model.bind_tools(tools)
agent_executor = create_react_agent(model, tools)